In [ ]:
import numpy as np
import pandas as pd
import pyodbc
import datetime as dt

#CREATE SERVER CONNECTION for postpaid
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=ServerName;'
                      'Database=DatabaseName;'
                      'UID=userName;'
                      'PWD=password;'
                      )

#CREATE SERVER CONNECTION fopr prepaid
ppmconn = pyodbc.connect('Driver={SQL Server};'
                      'Server=ServerName;'
                      'Database=DatabaseName;'
                      'UID=userName;'
                      'PWD=password;'
                      )

# Open and read Billing Details query file as a single buffer
bd = open('./Query/billing_details_query.sql', 'r')
bd_sqlFile = bd.read()

# Open and read payment query file as a single buffer
read_pmt = open('./Query/Payment_query.sql', 'r')
pmt_sqlFile = read_pmt.read()

# Open and read the adjustment file as a single buffer
read_adj = open('./Query/Adjustment_query.sql', 'r')
adj_sqlFile = read_adj.read()

# Open and read the ppm payment query file as a single buffer
read_ppm_payment = open('./Query/Payment_PPM.sql', 'r')
ppm_pmt_sqlFile = read_ppm_payment.read()

# Open and read the ppm BU query file as a single buffer
read_ppm_bu = open('./Query/PPM_Script.sql', 'r')
ppm_bu_sqlFile = read_ppm_bu.read()



sub_bill_details_df = pd.read_sql_query(bd_sqlFile, conn)
sub_payment_df = pd.read_sql_query(pmt_sqlFile, conn)
sub_adjustment_df = pd.read_sql_query(adj_sqlFile, conn)
ppm_payment_df = pd.read_sql_query(ppm_pmt_sqlFile, ppmconn)
ppm_bu_df= pd.read_sql_query(ppm_bu_sqlFile, ppmconn)

tariff_rate_df= sub_bill_details_df[['TariffClassName','TariffRates']]
tariff_rate_df1= tariff_rate_df.drop_duplicates(keep= 'first')
sub_ppm_bu_trf_df= pd.merge(ppm_bu_df, tariff_rate_df1, on='TariffClassName', how='inner')


sub_ppm_df= pd.merge(ppm_payment_df, sub_ppm_bu_trf_df, on= 'MeterNumber', how='inner')
sub_ppm_df["PaidAmount"].sum()
sub_bill_details_df['DisplayCode']= sub_bill_details_df['DisplayCode'].replace('NA', 'INACTIVE')
sub_bill_details_df['MeterNumber'] = sub_bill_details_df['MeterNumber'].apply(lambda x: "METERED" if not pd.isnull(x) else 'NOMETER')
sub_bd_pmt_combined_2 = pd.concat([sub_bill_details_df, sub_payment_df],axis=0)
pmt_adj_bd_combined_2= pd.concat([sub_bd_pmt_combined_2, sub_adjustment_df],axis=0)
pmt_adj_bd_combined_2['TariffDesc']= pmt_adj_bd_combined_2['TariffClassName']

unique_tariff_class_name= (pmt_adj_bd_combined_2['TariffClassName']).sort_values(ascending=True).unique() #get unique tariff names and Sort them alphabetically
unique_tariff_class_name_copy= unique_tariff_class_name.copy()# create a copy to get Tariff Description
pmt_adj_bd_combined_2['TariffDesc']= list(map(lambda x: x.replace('(A1S)','').replace('(A1T)','').replace('(C1S)','')
                                         .replace('(C1T)','').replace('(C2)','').replace('(D1T)','')
                                         .replace('(D2)','').replace('(R2S)','').replace('(R2T)','')
                                         .replace('(R3)','').replace('(R4)','').replace('(A2)','')
                                         .replace('(C3)','').replace('(R1)','').replace('(D1S)','')
                                         .replace('(S1)',''),pmt_adj_bd_combined_2['TariffDesc']))

pmt_adj_bd_combined_2[pmt_adj_bd_combined_2['TariffRates']==None]
(pmt_adj_bd_combined_2['TariffRates']).median()
pmt_adj_bd_combined_2['TariffRates']= (pmt_adj_bd_combined_2['TariffRates']).fillna(0.0)

#Delete thissub_bill_details_df
pd.options.display.float_format = '{:.2f}'.format
postpaid_by_tariff= pmt_adj_bd_combined_2.groupby(['BusinessUnitName', 'TariffDesc', 'TariffClassName']).apply(lambda x: pd.Series(dict(
    Active= ((x.DisplayCode == 'A').sum()+ (x.DisplayCode == 'R').sum()+ (x.DisplayCode == 'D').sum()+ (x.DisplayCode == 'M').sum()),
    In_Active= (x.DisplayCode == 'INACTIVE').sum(),
    Total= ((x.DisplayCode == 'INACTIVE').sum()+(x.DisplayCode == 'A').sum()+ (x.DisplayCode == 'R').sum()+ (x.DisplayCode == 'D').sum()+ (x.DisplayCode == 'M').sum()),
    Energy_Billed= (x.Energy_Billed).sum(),
    Revenue_Billed= (x.Revenue_Billed).sum(),
    BilledAmount= (x.BilledAmount).sum(),
    Metered= (x.MeterNumber=='METERED').sum(),
    Un_metered= (x.MeterNumber=='NOMETER').sum(),
    Total_= (x.MeterNumber=='METERED').sum() + (x.MeterNumber=='NOMETER').sum(),
    DIRECT=(x.DisplayCode == 'D').sum(),
    EST=(x.DisplayCode == 'A').sum(),
    MIN_FC=(x.DisplayCode == 'M').sum(),
    READ=(x.DisplayCode == 'R').sum(),
    No_of_Bills= ((x.DisplayCode == 'A').sum()+ (x.DisplayCode == 'R').sum()+ (x.DisplayCode == 'D').sum()+ (x.DisplayCode == 'M').sum()),
    RESP= (x.PmtAccountNumber).count(),
    Payment= (x.PaidAmount).sum(),
    Opening_Balance= (x.Opening_Balance).sum(),
    Closing_Balance= (x.Closing_Balance).sum(),
    Adjustment= (x.AmountEffected).sum(),
    TariffRate= (x.TariffRates).median(),
    
)))


sub_ppm_df['TariffDesc']= sub_ppm_df['TariffClassName']#unique_tariff_class_name_copy= unique_tariff_class_name.copy()# create a copy to get Tariff Description
sub_ppm_df['TariffDesc'] = list(map(lambda x: x.replace('(A1S)','').replace('(A1T)','').replace('(C1S)','')
                                         .replace('(C1T)','').replace('(C2)','').replace('(D1T)','')
                                         .replace('(D2)','').replace('(R2S)','').replace('(R2T)','')
                                         .replace('(R3)','').replace('(R4)','').replace('(A2)','')
                                         .replace('(C3)','').replace('(R1)','').replace('(D1S)','')
                                         .replace('(S1)',''),sub_ppm_df['TariffDesc']))

#Delete thissub_bill_details_df
pd.options.display.float_format = '{:.2f}'.format
prepaid_by_tariff= sub_ppm_df.groupby(['BusinessUnitName', 'TariffDesc', 'TariffClassName']).apply(lambda x: pd.Series(dict(
    Active= (x.DisplayCode == 'Active').sum(),
    In_Active= 0,
    Total= (x.DisplayCode == 'Active').sum(),
    Energy_Billed= (x.Energy_Billed).sum(),
    Revenue_Billed= (x.Revenue_Billed).sum(),
    BilledAmount= (x.BilledAmount).sum(),
    Metered= (x.MeterNumber).count(),
    Un_metered= 0,
    Total_= (x.MeterNumber).count(),
    DIRECT= 0,
    EST= 0,
    MIN_FC= 0,
    READ= (x.MeterNumber).count(),
    No_of_Bills= (x.MeterNumber).count(),
    RESP= (x.PaidAmount).count(),
    Payment= (x.PaidAmount).sum(),
    Opening_Balance= 0,
    Closing_Balance= 0,
    Adjustment= 0,
    TariffRate= (x.TariffRates).median()
    
)))

pmt_adj_bd_ppm_combined= pd.concat([pmt_adj_bd_combined_2, sub_ppm_df],axis=0)

#Delete thissub_bill_details_df
pd.options.display.float_format = '{:.2f}'.format
bs_by_customer_type= pmt_adj_bd_ppm_combined.groupby(['BusinessUnitName', 'ServiceUnitName', 'CustomerType']).apply(lambda x: pd.Series(dict(
    Active= ((x.DisplayCode == 'A').sum()+ (x.DisplayCode == 'Active').sum()+(x.DisplayCode == 'R').sum()+ (x.DisplayCode == 'D').sum()+ (x.DisplayCode == 'M').sum()),
    In_Active= (x.DisplayCode == 'INACTIVE').sum(),
    Total= ((x.DisplayCode == 'INACTIVE').sum()+(x.DisplayCode == 'A').sum()+ (x.DisplayCode == 'R').sum()+ (x.DisplayCode == 'D').sum()+ (x.DisplayCode == 'M').sum()),
    Energy_Billed= (x.Energy_Billed).sum(),
    Revenue_Billed= (x.Revenue_Billed).sum(),
    BilledAmount= (x.BilledAmount).sum(),
    Metered= (x.MeterNumber=='METERED').sum()+ (x.CustomerType=='PPM').sum(),
    Un_metered= (x.MeterNumber=='NOMETER').sum(),
    Total_= (x.MeterNumber=='METERED').sum() + (x.MeterNumber=='NOMETER').sum(),
    DIRECT=(x.DisplayCode == 'D').sum(),
    EST=(x.DisplayCode == 'A').sum(),
    MIN_FC=(x.DisplayCode == 'M').sum(),
    READ=(x.DisplayCode == 'R').sum(),
    No_of_Bills= ((x.DisplayCode == 'A').sum()+ (x.DisplayCode == 'R').sum()+ (x.DisplayCode == 'D').sum()+ (x.DisplayCode == 'M').sum()),
    RESP= (x.PmtAccountNumber).count(),
    Payment= (x.PaidAmount).sum(),
    Opening_Balance= (x.Opening_Balance).sum(),
    Closing_Balance= (x.Closing_Balance).sum(),
    Adjustment= (x.AmountEffected).sum()
    
)))

list_of_bu= sub_bill_details_df['BusinessUnitName'].dropna().sort_values(ascending=True).unique()
list_of_bu
number_of_bu= len(list_of_bu)

#Creating a sub for sheet 1
consolidated_billing_summary_by_customer_type= bs_by_customer_type.reset_index()
#consolidated_billing_summary_by_customer_type.rename(columns=dict, inplace=True)

#Creating a sub for sheet 2
postpaid_billing_summary_by_tariff= postpaid_by_tariff.reset_index()
#postpaid_billing_summary_by_tariff.rename(columns=dict, inplace=True)

#FOR SHEET 3
cons_ppm_billing_summary_by_tariff= prepaid_by_tariff.reset_index()
#cons_ppm_billing_summary_by_tariff.rename(columns=dict, inplace=True)

#This generates report for indiovidual BUs 
for bu in range(number_of_bu):
    buName= list_of_bu[bu]
    #consolidated_first_sheet= billing_summary_by_customer_type_df[billing_summary_by_customer_type_df['BusinessUnitName']==list_of_bu[buName]]
    
    consolidated_first_sheet= consolidated_billing_summary_by_customer_type[consolidated_billing_summary_by_customer_type['BusinessUnitName']==buName]
    
    consolidated_second_sheet= postpaid_billing_summary_by_tariff[postpaid_billing_summary_by_tariff['BusinessUnitName']==buName]
    consolidated_third_sheet= cons_ppm_billing_summary_by_tariff[cons_ppm_billing_summary_by_tariff['BusinessUnitName']==buName]
    
    with pd.ExcelWriter(buName + ' Billing_Status v2.0.xlsx') as writer:  
            consolidated_first_sheet.to_excel(writer, sheet_name='BILLING_SUMMARY_BY_CUSTOMER_TYP', index=False, index_label='SNO')
            consolidated_second_sheet.to_excel(writer, sheet_name='POST_BILLING_SUMMARY_BY_TARIFF', index=False, index_label='SNO')
            consolidated_third_sheet.to_excel(writer, sheet_name='PPM_SUMMARY_BY_TARIFF', index=False, index_label='SNO')
#print(bu_only_report_df)

# THIS GENERATES CONSOLIDATED FILE
with pd.ExcelWriter('Consolidated Billing_Status v2.xlsx') as writer:  
            consolidated_billing_summary_by_customer_type.to_excel(writer, sheet_name='BILLING_SUMMARY_BY_CUSTOMER_TYP', index=False, index_label='SNO')
            postpaid_billing_summary_by_tariff.to_excel(writer, sheet_name='POST_BILLING_SUMMARY_BY_TARIFF', index=False, index_label='SNO')
            cons_ppm_billing_summary_by_tariff.to_excel(writer, sheet_name='PPM_SUMMARY_BY_TARIFF', index=False, index_label='SNO')
#print(bu_only_report_df)



pmt_adj_bd_ppm_combined['Disco']='BEDC'
pmt_adj_bd_combined_2['Disco']='BEDC'
sub_ppm_df['Disco']='BEDC'

#sheet 1
pd.options.display.float_format = '{:.2f}'.format
disco_by_customer_type= pmt_adj_bd_ppm_combined.groupby(['Disco','CustomerType']).apply(lambda x: pd.Series(dict(
    Active= ((x.DisplayCode == 'A').sum()+ (x.DisplayCode == 'Active').sum()+(x.DisplayCode == 'R').sum()+ (x.DisplayCode == 'D').sum()+ (x.DisplayCode == 'M').sum()),
    In_Active= (x.DisplayCode == 'INACTIVE').sum(),
    Total= ((x.DisplayCode == 'INACTIVE').sum()+(x.DisplayCode == 'Active').sum()+(x.DisplayCode == 'A').sum()+ (x.DisplayCode == 'R').sum()+ (x.DisplayCode == 'D').sum()+ (x.DisplayCode == 'M').sum()),
    Energy_Billed= (x.Energy_Billed).sum(),
    Revenue_Billed= (x.Revenue_Billed).sum(),
    BilledAmount= (x.BilledAmount).sum(),
    Metered= (x.MeterNumber=='METERED').sum()+ (x.CustomerType=='PPM').sum(),
    Un_metered= (x.MeterNumber=='NOMETER').sum(),
    Total_= (x.MeterNumber=='METERED').sum() + (x.MeterNumber=='NOMETER').sum(),
    DIRECT=(x.DisplayCode == 'D').sum(),
    EST=(x.DisplayCode == 'A').sum(),
    MIN_FC=(x.DisplayCode == 'M').sum(),
    READ=(x.DisplayCode == 'R').sum(),
    No_of_Bills= ((x.DisplayCode == 'A').sum()+ (x.DisplayCode == 'R').sum()+ (x.DisplayCode == 'D').sum()+ (x.DisplayCode == 'M').sum()),
    RESP= (x.PmtAccountNumber).count(),
    Payment= (x.PaidAmount).sum(),
    Opening_Balance= (x.Opening_Balance).sum(),
    Closing_Balance= (x.Closing_Balance).sum(),
    Adjustment= (x.AmountEffected).sum()
    
)))



#Delete thissub_bill_details_df
pd.options.display.float_format = '{:.2f}'.format
disco_billing_summary_by_tariff= pmt_adj_bd_combined_2.groupby(['Disco', 'TariffDesc', 'TariffClassName']).apply(lambda x: pd.Series(dict(
    Active= ((x.DisplayCode == 'A').sum()+ (x.DisplayCode == 'R').sum()+ (x.DisplayCode == 'D').sum()+ (x.DisplayCode == 'M').sum()),
    In_Active= (x.DisplayCode == 'INACTIVE').sum(),
    Total= ((x.DisplayCode == 'INACTIVE').sum()+(x.DisplayCode == 'A').sum()+ (x.DisplayCode == 'R').sum()+ (x.DisplayCode == 'D').sum()+ (x.DisplayCode == 'M').sum()),
    Energy_Billed= (x.Energy_Billed).sum(),
    Revenue_Billed= (x.Revenue_Billed).sum(),
    BilledAmount= (x.BilledAmount).sum(),
    Metered= (x.MeterNumber=='METERED').sum(),
    Un_metered= (x.MeterNumber=='NOMETER').sum(),
    Total_= (x.MeterNumber=='METERED').sum() + (x.MeterNumber=='NOMETER').sum(),
    DIRECT=(x.DisplayCode == 'D').sum(),
    EST=(x.DisplayCode == 'A').sum(),
    MIN_FC=(x.DisplayCode == 'M').sum(),
    READ=(x.DisplayCode == 'R').sum(),
    No_of_Bills= ((x.DisplayCode == 'A').sum()+ (x.DisplayCode == 'R').sum()+ (x.DisplayCode == 'D').sum()+ (x.DisplayCode == 'M').sum()),
    RESP= (x.PmtAccountNumber).count(),
    Payment= (x.PaidAmount).sum(),
    Opening_Balance= (x.Opening_Balance).sum(),
    Closing_Balance= (x.Closing_Balance).sum(),
    Adjustment= (x.AmountEffected).sum()
    
)))


#Delete thissub_bill_details_df
pd.options.display.float_format = '{:.2f}'.format
disco_ppm_billing_summary_by_tariff= sub_ppm_df.groupby(['Disco', 'TariffDesc', 'TariffClassName']).apply(lambda x: pd.Series(dict(
    Active= (x.DisplayCode == 'Active').sum(),
    In_Active= 0,
    Total= (x.DisplayCode == 'Active').sum(),
    Energy_Billed= (x.Energy_Billed).sum(),
    Revenue_Billed= (x.Revenue_Billed).sum(),
    BilledAmount= (x.BilledAmount).sum(),
    Metered= (x.MeterNumber).count(),
    Un_metered= 0,
    Total_= (x.MeterNumber).count(),
    DIRECT= 0,
    EST= 0,
    MIN_FC= 0,
    READ= (x.MeterNumber).count(),
    No_of_Bills= (x.MeterNumber).count(),
    RESP= (x.PaidAmount).count(),
    Payment= (x.PaidAmount).sum(),
    Opening_Balance= 0,
    Closing_Balance= 0,
    Adjustment= 0
    
)))



#Creating a sub for sheet 1
diso_consolidated_billing_summary_by_customer_type= disco_by_customer_type.reset_index()
#Creating a sub for sheet 2
disco_postpaid_billing_summary_by_tariff= disco_billing_summary_by_tariff.reset_index()
disco_cons_ppm_billing_summary_by_tariff= disco_ppm_billing_summary_by_tariff.reset_index()

#THIS GENERATES BENIN DISCO CONSOLIDATED FILE
with pd.ExcelWriter('BENIN DISCO v2 Billing_Status .xlsx') as writer:  
            diso_consolidated_billing_summary_by_customer_type.to_excel(writer, sheet_name='BILLING_SUMMARY_BY_CUSTOMER_TYP', index=False, index_label='SNO')
            disco_postpaid_billing_summary_by_tariff.to_excel(writer, sheet_name='POST_BILLING_SUMMARY_BY_TARIFF', index=False, index_label='SNO')
            disco_cons_ppm_billing_summary_by_tariff.to_excel(writer, sheet_name='PPM_SUMMARY_BY_TARIFF', index=False, index_label='SNO')
#print(bu_only_report_df)
